In [3]:
val PATH = "file:///Users/lzz/work/SparkML/"

import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.mllib.linalg.Vectors

// Load and parse the data file.
    val rows = sc.textFile( PATH+"data/mllib/sample_lda_data.txt").map { line =>
      val values = line.split(' ').map(_.toDouble)
      Vectors.dense(values)
    }
    val mat = new RowMatrix(rows)

    // Compute SVD.
    val svd = mat.computeSVD(mat.numCols().toInt)

    println("Singular values are " + svd.s)


Singular values are [25.26776724352383,13.752505639532256,12.193628505511972,8.073436745030213,7.4973616664530764,6.27744176820788,5.089006966420011,2.153286728359266,1.3722823724537987,0.7112974499172692,0.055896633645633485]
